# <font color='lightblue'>Final project - sentiment analysis comparison part 2</font>

#Imports & functions

In [ ]:
import pandas as pd 
import numpy as np
import math
import string 
#connect to drive
from google.colab import drive
# visualisation
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt 
# svm
from sklearn import svm
# naive bayes
from sklearn.naive_bayes import GaussianNB
#XGBoost
from sklearn.ensemble import GradientBoostingClassifier
#accuracy precision recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
#grid search with cross validation
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
#datetime for timer
from datetime import datetime # for timer method

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

In [ ]:
def round_ths(x,ths):
  if x<ths :
    return 0
  else :
    return 1

In [ ]:
def create_df_of_results(y_test,prob_y_pred):  
  results = pd.DataFrame()
  results['target'] = y_test
  results['prob_1'] = prob_y_pred[:,1]
  results['pred_ths_01']  =  results.apply(lambda x: round_ths(x['prob_1'],0.1),axis=1)
  results['pred_ths_02']  =  results.apply(lambda x: round_ths(x['prob_1'],0.2),axis=1)
  results['pred_ths_03']  =  results.apply(lambda x: round_ths(x['prob_1'],0.3),axis=1)
  results['pred_ths_04']  =  results.apply(lambda x: round_ths(x['prob_1'],0.4),axis=1)
  results['pred_ths_05']  =  results.apply(lambda x: round_ths(x['prob_1'],0.5),axis=1)
  results['pred_ths_06']  =  results.apply(lambda x: round_ths(x['prob_1'],0.6),axis=1)
  results['pred_ths_07'] =   results.apply(lambda x: round_ths(x['prob_1'],0.7),axis=1)
  results['pred_ths_08'] =   results.apply(lambda x: round_ths(x['prob_1'],0.8),axis=1)
  results['pred_ths_09'] =   results.apply(lambda x: round_ths(x['prob_1'],0.9),axis=1)
  return results

In [ ]:
def convert_results_df_to_scores_per_ths(results): 
  scores_test = pd.DataFrame(columns = ["ths","accuracy","recall","precision"])
  i=0.1
  for (columnName, columnData) in results[['pred_ths_01','pred_ths_02','pred_ths_03','pred_ths_04','pred_ths_05','pred_ths_06','pred_ths_07','pred_ths_08','pred_ths_09']].iteritems():
    scores_temp={'ths':i , 'accuracy': accuracy_score(results['target'], results[columnName]), 'recall':recall_score(results['target'], results[columnName]),'precision':precision_score(results['target'], results[columnName])}# add score of each cross val to df
    scores_test=scores_test.append(scores_temp, ignore_index = True)
    # if (columnName == 'y_pred_05'):
    #   i=i+0.4
    # else:
    #   if (columnName == 'y_pred_099'):
    #     i=i+0.005
    #   else:
    #     i=i+0.01
    i=i+0.1
    
  return scores_test

In [ ]:
def create_graph_for_results_metrics_per_ths(y_pred_prob,y_test,plot_title='Results of the model - test dataset'):
  res=create_df_of_results(y_test,y_pred_prob)
  scores_per_ths=convert_results_df_to_scores_per_ths(res)
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=scores_per_ths['ths'], y=scores_per_ths['accuracy'],line=dict(color=px.colors.qualitative.Pastel[0], width=4),
                      mode='lines+markers',
                      name='accuracy'))
  fig.add_trace(go.Scatter(x=scores_per_ths['ths'], y=scores_per_ths['recall'],line=dict(color=px.colors.qualitative.Pastel[1], width=4),
                      mode='lines+markers',
                      name='recall'))
  fig.add_trace(go.Scatter(x=scores_per_ths['ths'], y=scores_per_ths['precision'],line=dict(color=px.colors.qualitative.Pastel[2], width=4),
                      mode='lines+markers', name='precision'))
  fig.update_layout(title=plot_title,
                     xaxis_title='Threshold of confidence',
                     )
  fig.show()

# Load data

## train

In [ ]:
# data is from : https://www.kaggle.com/datasets/therohk/ireland-historical-news?datasetId=30661
drive.mount('/content/drive')
path = "/content/drive/MyDrive/final project/data_files/train_ml.csv"
train = pd.read_csv(path)
train.drop(['Unnamed: 0'], axis='columns', inplace=True)
train.drop(['label.1'], axis='columns', inplace=True)
print(train.shape)
train.head()

Mounted at /content/drive
(40000, 9)


,label,count_of_words,percent_of_ner,percent_of_punc,percent_of_upper_case,percent_of_nouns,percent_of_verbs,percent_of_adj,positve_negative_words_ratio
0,0.0,0.159444,0.080090,0.118815,0.024538,0.150423,0.501269,0.152284,0.001472
1,0.0,0.111611,0.136703,0.078550,0.024099,0.145244,0.487365,0.134777,0.001639
2,1.0,0.036386,0.101792,0.049628,0.022276,0.162284,0.349462,0.143369,0.099975
3,0.0,0.069501,0.145083,0.125047,0.070115,0.157980,0.402299,0.145594,0.000222
4,0.0,0.118561,0.160998,0.141287,0.049326,0.123514,0.527211,0.154195,0.001222


In [ ]:
y_train=train['label']
X_train=train.drop(['label'], axis='columns')

## test

In [ ]:
path = "/content/drive/MyDrive/final project/data_files/test_ml.csv"
test = pd.read_csv(path)
test.drop(['Unnamed: 0'], axis='columns', inplace=True)
test.drop(['label.1'], axis='columns', inplace=True)


In [ ]:
y_test=test['label']
X_test=test.drop(['label'], axis='columns')

# SVM

In [ ]:
svm_model = svm.SVC(kernel='linear',probability=True) # Linear Kernel
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
#get scores
print("accuracy score :",accuracy_score(y_test, y_pred))
print("recall score :",recall_score(y_test, y_pred))
print("precision score :",precision_score(y_test, y_pred))

accuracy score : 0.6492
recall score : 0.6977574915657868
precision score : 0.6391565169969097


precision: tp/p, when we say item is positive, what are the odds it's actually positive?

recall: tp/t how many positive items we missed = 1- recall


In [ ]:
prob_y_pred=svm_model.predict_proba(X_test)

In [ ]:
create_graph_for_results_metrics_per_ths(prob_y_pred,y_test)

<ipython-input-30-803eebbbaf7a>:4: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-inpu

In [ ]:
svm_model = svm.SVC(kernel='linear',C=0.3) # Linear Kernel with strong regularization
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
#get scores
print("accuracy score :",accuracy_score(y_test, y_pred))
print("recall score :",recall_score(y_test, y_pred))
print("precision score :",precision_score(y_test, y_pred))

accuracy score : 0.6375
recall score : 0.7299067275253026
precision score : 0.6189835072366207


In [ ]:
svm_model =svm.NuSVC(gamma="auto")#non linear kernel
svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
#get scores
print("accuracy score :",accuracy_score(y_test, y_pred))
print("recall score :",recall_score(y_test, y_pred))
print("precision score :",precision_score(y_test, y_pred))

accuracy score : 0.5625
recall score : 0.6015082357610637
precision score : 0.5615042608373472


#Gaussian Naive Bayes

In [ ]:
gnb = GaussianNB()
nb_model=gnb.fit(X_train, y_train)
y_pred =nb_model.predict(X_test)
#get scores
print("accuracy score :",accuracy_score(y_test, y_pred))
print("recall score :",recall_score(y_test, y_pred))
print("precision score :",precision_score(y_test, y_pred))

accuracy score : 0.5047
recall score : 0.990672752530264
precision score : 0.5043443119822186


In [ ]:
prob_y_pred=nb_model.predict_proba(X_test)

In [ ]:
create_graph_for_results_metrics_per_ths(prob_y_pred,y_test)

<ipython-input-30-803eebbbaf7a>:4: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-inpu

#XGBoost

## CV grid search hyperparams optimization

In [ ]:
# A parameter grid for XGBoost
params = {
        'min_samples_split': [2, 5, 7,10],
        'max_depth': [3, 4, 5,6],
        'n_estimators': [50,100,200,300],
        'learning_rate': [0.1,0.2,0.3,0.5]
        }

In [ ]:
xgb = GradientBoostingClassifier()

In [ ]:
# CV for searching optimize params
folds = 5
param_comb = 4

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 42)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='accuracy', n_jobs=4, cv=skf.split(X_train,y_train), verbose=3, random_state=42 )

# Here we go
start_time = timer(None) # timing starts from this point for "start_time" variable
random_search.fit(X_train, y_train)
timer(start_time) # timing ends here for "start_time" variable

Fitting 5 folds for each of 4 candidates, totalling 20 fits

 Time taken: 0 hours 5 minutes and 6.35 seconds.


In [ ]:
print('\n Best hyperparameters:')
print(random_search.best_params_)


 Best hyperparameters:
{'n_estimators': 200, 'min_samples_split': 5, 'max_depth': 3, 'learning_rate': 0.1}


In [ ]:
results = pd.DataFrame(random_search.cv_results_)
results.head(20)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_max_depth,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,22.613356,1.037179,0.038526,0.007741,50,5,5,0.5,"{'n_estimators': 50, 'min_samples_split': 5, '...",0.700500,0.694625,0.706875,0.710250,0.696250,0.701700,0.006018,4
1,53.377000,1.225427,0.086805,0.027049,200,5,3,0.1,"{'n_estimators': 200, 'min_samples_split': 5, ...",0.711875,0.707375,0.718875,0.724875,0.712250,0.715050,0.006132,1
2,82.057799,3.405968,0.134909,0.034969,300,10,3,0.2,"{'n_estimators': 300, 'min_samples_split': 10,...",0.714375,0.705375,0.715750,0.723500,0.709875,0.713775,0.006076,2
3,49.100855,10.081583,0.060960,0.035595,200,10,3,0.5,"{'n_estimators': 200, 'min_samples_split': 10,...",0.704375,0.697125,0.705750,0.719625,0.705875,0.706550,0.007289,3


## predict

In [ ]:
XGB_clf = GradientBoostingClassifier(n_estimators=200,min_samples_split=5, learning_rate=0.1,max_depth=3, random_state=0)
XGB_model=XGB_clf.fit(X_train, y_train)
y_pred=XGB_model.predict(X_test)
#get scores
print("accuracy score :",accuracy_score(y_test, y_pred))
print("recall score :",recall_score(y_test, y_pred))
print("precision score :",precision_score(y_test, y_pred))

accuracy score : 0.6504
recall score : 0.843818217900377
precision score : 0.6108317770435282


In [ ]:
prob_y_pred=XGB_model.predict_proba(X_test)

In [ ]:
print("first column: ",round(XGB_clf.classes_[0]),"second column: ",round(XGB_clf.classes_[1]))  #check order of the predict proba results and that it fits the function

first column:  0 second column:  1


In [ ]:
create_graph_for_results_metrics_per_ths(prob_y_pred,y_test)

<ipython-input-30-803eebbbaf7a>:4: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-input-30-803eebbbaf7a>:6: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

<ipython-inpu